# Hội thi Thử thách Trí tuệ Nhân tạo Thành phố Hồ Chí Minh 2024

## Hướng dẫn truy vấn dữ liệu thị giác dùng fiftyone

Đây là hướng dẫn dùng cho các đội tham dự AI Challenge 2024. Hướng dẫn này nhằm mục đích giới thiệu cho các đội một phương pháp cơ bản để truy vấn dữ liệu dựa trên thông tin BTC cung cấp và giới thiệu công cụ fiftyone để hỗ trợ đội thi đánh giá kết quả.


In [ ]:
! pip install faiss-cpu

In [ ]:
# ! pip install fiftyone==0.24.1
# ! pip install torch torchvision torchaudio

In [ ]:
# ! pip install torch
# ! pip install ftfy regex tqdm
# ! pip install git+https://github.com/openai/CLIP.git

## Define library

In [ ]:
import fiftyone as fo
import fiftyone.brain as fob
import numpy as np
from glob import glob
import json 
import os
import torch
import open_clip

In [ ]:
import faiss

Load dữ liệu keyframe từ thư mục chứa keyframe. Trong hướng dẫn này tất cả các file Keyframes_L*.zip được giải nén vào thư mục `D:\AIC\Keyframes`. Mỗi ảnh và thông tin đi kèm sau này sẽ được lưu trữ trong một `Sample`. Tất cả các `Sample` được lưu trong `Dataset`.

In [ ]:
dataset = fo.Dataset.from_images_dir('/home/nguyenhoangphuc-22521129/AIC/Keyframes_L01/keyframes', name=None, tags=None, recursive=True)

Sau khi dữ liệu đã load lên xong. Bạn có thể truy cập vào đường vào ứng dụng web của fiftyone từ [http://localhost:5151](http://localhost:5151)

In [ ]:
session = fo.launch_app(dataset, auto=False)

In [ ]:
ls /home/nguyenhoangphuc-22521129/AIC/Keyframes_L01/keyframes

## Hoặc chạy cell này để mở tab cho ứng dụng web fiftyone

In [ ]:
session.open_tab()

## Trích xuất thêm thông tin tên của video và frameid
Thông tin `video` và `frameid` sẽ được lấy từ tên của tập tin keyframe.

In [ ]:
#dataset

In [ ]:
for sample in dataset:
    _, sample['video'], sample['frameid'] = sample['filepath'][:-4].rsplit('/', 2)
    sample.save()

In [ ]:
print(dataset.first())

## Thêm thông tin kết quả của object detection.

Bước này có thể tốn của bạn nhiều thời gian để đọc hết tất cả các dữ liệu về object detection. Bạn có thể bỏ qua cell này và chạy cell này sau nếu muốn thử thêm các thông tin về vector CLIP embedding trước.

In [ ]:
for sample in dataset:
    object_path = f"/home/nguyenhoangphuc-22521129/AIC/HCMAI22_MiniBatch1/Objects/{sample['video']}/{sample['frameid']}.json"
    with open(object_path) as jsonfile:
        det_data = json.load(jsonfile)
    detections = []
    for cls, box, score in zip(det_data['detection_class_entities'], det_data['detection_boxes'], det_data['detection_scores']):
        # Convert to [top-left-x, top-left-y, width, height]
        boxf = [float(box[1]), float(box[0]), float(box[3]) - float(box[1]), float(box[2]) - float(box[0])]
        scoref = float(score)

        # Only add objects with confidence > 0.4
        if scoref > 0.4:
            detections.append(
                fo.Detection(
                    label=cls,
                    bounding_box= boxf,
                    confidence=float(score)
                )
            )
    sample["object_faster_rcnn"] = fo.Detections(detections=detections)
    sample.save()


## Thêm thông tin CLIP embedding.

In [ ]:
all_keyframe = glob('/home/nguyenhoangphuc-22521129/AIC/Keyframes_L01/keyframes/*/*.jpg')
video_keyframe_dict = {}
all_video = glob('/home/nguyenhoangphuc-22521129/AIC/Keyframes_L01/keyframes/*')
all_video = [v.rsplit('/',1)[-1] for v in all_video]

Tạo dictionary `video_keyframe_dict` với `video_keyframe_dict[video]` thông tin danh sách `keyframe` của `video`

In [ ]:
for kf in all_keyframe:
    _, vid, kf = kf[:-4].rsplit('/',2)
    if vid not in video_keyframe_dict.keys():
        video_keyframe_dict[vid] = [kf]
    else:
        video_keyframe_dict[vid].append(kf)

Do thông tin vector CLIP embedding được cung cấp được lưu theo từng video nhầm mục đích tối ưu thời gian đọc dữ liệu. Cần sort lại danh sách `keyframe` của từng `video` để đảm bảo thứ tự đọc đúng với vector embedding được cung cấp.

In [ ]:
for k,v in video_keyframe_dict.items():
    video_keyframe_dict[k] = sorted(v)

Tạo dictionary `embedding_dict` với `embedding_dict[video][keyframe]` lưu thông tin vector CLIP embedding của `keyframe` trong `video` tương ứng

In [ ]:
embedding_dict = {}
for v in all_video:
    clip_path = f'/home/nguyenhoangphuc-22521129/AIC/clip-features-vit-b32-sample/clip-features-vit-b32-sample/clip-features/{v}.npy'
    a = np.load(clip_path)
    embedding_dict[v] = {}
    for i,k in enumerate(video_keyframe_dict[v]):
        embedding_dict[v][k] = a[i]


Tạo danh sách `clip_embedding` ứng với danh sách `sample` trong `dataset`.

In [ ]:
clip_embeddings = []
for sample in dataset:
    clip_embedding = embedding_dict[sample['video']][sample['frameid']]
    sample['clip_embedding'] = clip_embedding
    sample.save()
    clip_embeddings.append(clip_embedding)


In [ ]:
embedding_dict.keys()

In [ ]:
model_and_preprocess = open_clip.create_model_and_transforms("ViT-B-16", pretrained="openai")


In [ ]:
model = model_and_preprocess[0]

In [ ]:
preprocess = model_and_preprocess[1]

In [ ]:
model.eval()

In [ ]:
input_resolution = preprocess.transforms[0].size[0]  # Kích thước đầu vào
context_length = model.context_length
vocab_size = model.vocab_size

In [ ]:
text_search = "A female is drawing a landscape picture"
text_tokens = open_clip.tokenize([text_search])
with torch.no_grad():
    text_features = model.encode_text(text_tokens).float()

In [ ]:
for sample in dataset:
    a = sample['clip_embedding']
    query_similarity = (text_features @ a.reshape(1,512).T).cpu().numpy().item()
    sample['text_query_similarity'] = query_similarity
    sample.save()

In [ ]:
import matplotlib.pyplot as plt

## Tạo chỉ mục Faiss

In [ ]:
d = clip_embeddings[0].shape[0]  # Kích thước vector
index = faiss.IndexFlatL2(d)  # Chỉ mục Flat với khoảng cách L2

In [ ]:
# Thêm các embedding vào chỉ mục
index.add(np.array(clip_embeddings).astype('float32'))

In [ ]:
# Giả sử bạn có một embedding truy vấn
query_embedding = np.random.random((1, d)).astype('float32')  # Thay bằng embedding thực tế

In [ ]:
# Tìm kiếm k vector gần nhất
k = 5  # Số lượng kết quả muốn tìm
distances, indices = index.search(query_embedding, k)

In [ ]:
# In ra kết quả
print("Khoảng cách đến các vector gần nhất:", distances)
print("Chỉ số của các vector gần nhất:", indices)

In [ ]:
dataset

In [ ]:
fob.compute_similarity(
    dataset,
    model="clip-vit-base32-torch",      # store model's name for future use
    embeddings=clip_embeddings,          # precomputed image embeddings
    brain_key="new_img_sim",
)

## Từ đây các bạn có thể thử các tính năng search, filter trên ứng dụng fiftyone.

In [ ]:
! pip install umap-learn

In [ ]:
# Bạn cần phải cài version umap-learn hỗ trợ.
fob.compute_visualization(
    dataset,
    embeddings=clip_embeddings,
    brain_key="img_viz"
)
